In [1]:
# !pip install dateparser
# !pip install geopy
# !pip install geonamescache
import numpy as np
import pandas as pd
from flask import Flask, jsonify
import requests
import pprint as pp
import time

from dateparser.search import search_dates
import datetime as dt

# Weather API
import openweathermapy.core as owm
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from geopy import geocoders  
gn = geocoders.GeoNames(username='moalarcon')
import geonamescache
from geonamescache.mappers import country

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# CHAMPS Travel Routes API call

# API call to GLOBE.gov for historical global weather

## Format Kayak response for GLOBE API call parameters

##### NOTE: CODE DOESNT WORK
Need to debug how to parse the string response from Kayak Scraping because the format of output messes up the parsing command. For the purposes of moving on, ignoring next cell and hard coding Kayak response.


In [ ]:
# #### SAMPLE RESPONSE FROM KAYAK: string type
# answer = 'Start from MiamiTue Mar 3,\xa04:45 PM10h 35mMadridStay 3 nightsSat Mar 7,\xa06:05 PM2h 05mParisStay 5 nightsThu Mar 12,\xa06:05 PM8h 25mNew YorkStay 4 nightsMon Mar 16,\xa03:37 PM3h 21mEnd in Miami'

# ## from kayak output parse: start, 3 destinations and end (includes city and dates)
# ans_parse = answer.split(",")
# ans_parse

# start = ans_parse[0]
# dest1 = ans_parse[1]
# dest2 = ans_parse[2]
# dest3 = ans_parse[3]
# end = ans_parse[4]

# start, dest1, dest2, dest3, end

# ## from variables above, parse city and dates into their own variables 
# ## (e.g., issue: non-standard formatting; see dest3)

# start_city = f"{start[2]}"
# start_date = f"{start[3]} {start[4]}"
# start_city
# start_date

# dest1 = dest1.split()
# dest1_city= f"{dest1[2]}" 
# dest1_date= f"{dest1[5]} {dest1[6]}"


# dest2 = dest2.split()
# dest2_city= f"{dest2[2]}" 
# dest2_date= f"{dest2[5]} {dest2[6]}"

# dest3 = dest3.split()
# dest3_city= f"{dest3[2]} {dest3[3]}" 
# dest3_date= f"{dest3[6]} {dest3[7]}"


NOTE: (amongst other things) would need to convert dates from Kayak scraping of dates from strings to datetime object to get format "MM-DD"
    e.g.: start_date = "Mar 3" to "03-03" AND end_date = "Mar 16" to "03-16"
Code runs into error "ValueError: time data 'Mar 3' does not match format '%b-%d'" because day is not zero-padded
    startdate = datetime.strptime(start_date,'%m-%d')
    enddate = datetime.strptime(end_date,'%m-%d')


### Set up API call parameters

NOTE: Using manual input parameters because parsing kayak response isn't working

In [2]:
## Combine CITIES and DATES for start, end and destinations into nested dictionary

weather_params = {"travel_details" : 
                  {1: {"city": "Madrid", "traveldates_start": "03-03", "traveldates_end": "03-06"}, 
                  2: {"city": "Paris", "traveldates_start": "03-07", "traveldates_end": "03-11"}, 
                  3: {"city": "New York City", "traveldates_start": "03-12", "traveldates_end": "03-15"}}}

cities = ["Madrid", "Paris", "New York City"]

In [ ]:
baseURL = "https://api.globe.gov/search/v1/measurement/protocol/measureddate/point/distance/?protocols=air_temp_dailies&protocols=humidity_noons&protocols=precipitations&protocols=sky_conditions"

for j in range(1,4):
    travel_start = weather_params["travel_details"][j]["traveldates_start"]
    travel_end = weather_params["travel_details"][j]["traveldates_end"]  
    start_date = f"2018-{travel_start}"   ##format for query
    end_date = f"2018-{travel_end}"   ##format for query
    city = weather_params["travel_details"][j]["city"]
    location = gn.geocode(city) 
    time.sleep(5)
    lat = location.latitude
    lon = location.longitude
#     print(city, travel_start, travel_end, lat, lon)
    query = f"{baseURL}&startdate={start_date}&enddate={end_date}&lat={lat}&lon={lon}&distanceinkm=75&geojson=FALSE&sample=FALSE"
    response = requests.get(f'{query}').json()
    JSON_ = response["results"]   ##LIST of dicts
    j = j+1
    
    print(f"---------------{city}---------------")
    print(city, travel_start, travel_end, lat, lon, query, response)

In [ ]:
# gc = geonamescache.GeonamesCache()
# countries = gc.get_countries()
# mapper = country(from_key='name', to_key='iso3')



# for city in cities:
#     city.geocode(city, exactly_one=True, timeout=DEFAULT_SENTINEL, country=None, country_bias=None)
#     gc.get_cities(city)
#     gc.get_countries(city)
#     print(city, travel_start, travel_end, lat, lon)


Eventually need to define function that  iterate through weather_params dict, pull values for query and run query


In [3]:
# Initialize list for output of API call;

baseURL = "https://api.globe.gov/search/v1/measurement/protocol/measureddate/point/distance/?protocols=air_temp_dailies&protocols=humidity_noons&protocols=precipitations&protocols=sky_conditions"

#initialize empty lists to append necessary weather data
date = []
max_temp = []
min_temp = []
precip = []
humidity = []
latitude = []
longitude= []
JSON_list = []

i=0

## iterrate through weather_params dict to pull out query parameters
for j in range(1,4):
    travel_start = weather_params["travel_details"][j]["traveldates_start"]
    travel_end = weather_params["travel_details"][j]["traveldates_end"]  
    start_date = f"2018-{travel_start}"
    end_date = f"2018-{travel_end}"
    city = weather_params["travel_details"][j]["city"]
    location = gn.geocode(city) 
    time.sleep(5)
    lat = location.latitude
    lon = location.longitude
    #API query URL
    query = f"{baseURL}&startdate={start_date}&enddate={end_date}&lat={lat}&lon={lon}&distanceinkm=100&geojson=FALSE&sample=FALSE"
    response = requests.get(f'{query}').json()
    JSON_ = response["results"]   ##LIST of dicts
    #     print(JSON_)

    for i in range(len(JSON_)): ## iterate through JSON_ list for each item[i], which are dicts

        ## create conditional to pull data from response    
        if JSON_[i]['protocol'] == 'air_temp_dailies':
                    ### Append JSON response values to lists for each output variable
            date.append(JSON_[i]['measuredDate'])
            max_temp.append(JSON_[i]['data']['airtempdailiesMaximumTemp'])
            min_temp.append(JSON_[i]['data']['airtempdailiesMinimumTemp'])
            latitude.append(JSON_[i]['latitude'])
            longitude.append(JSON_[i]['longitude'])
        elif JSON_[i]['protocol'] == 'humidity_noons':
            humidity.append(JSON_[i]['data']['humiditynoonsRelativeHumidityPercent'])
#             j=j+1
            i=i+1
        elif JSON_[i]['protocol'] == 'precipitations':
            precip.append(JSON_[i]['data']['precipitationsOccurrenceType'])
#             j=j+1
            i=i+1
        else: 
            date.append(np.nan)
            max_temp.append(np.nan)
            min_temp.append(np.nan)            
            latitude.append(np.nan)
            longitude.append(np.nan)
            humidity.append(np.nan)            
            precip.append(np.nan)
            i=i+1
    del JSON_
        
    print(city, travel_start, travel_end, lat, lon, query)

Madrid 03-03 03-06 40.4165 -3.70256 https://api.globe.gov/search/v1/measurement/protocol/measureddate/point/distance/?protocols=air_temp_dailies&protocols=humidity_noons&protocols=precipitations&protocols=sky_conditions&startdate=2018-03-03&enddate=2018-03-06&lat=40.4165&lon=-3.70256&distanceinkm=100&geojson=FALSE&sample=FALSE
Paris 03-07 03-11 48.85341 2.3488 https://api.globe.gov/search/v1/measurement/protocol/measureddate/point/distance/?protocols=air_temp_dailies&protocols=humidity_noons&protocols=precipitations&protocols=sky_conditions&startdate=2018-03-07&enddate=2018-03-11&lat=48.85341&lon=2.3488&distanceinkm=100&geojson=FALSE&sample=FALSE
New York City 03-12 03-15 40.71427 -74.00597 https://api.globe.gov/search/v1/measurement/protocol/measureddate/point/distance/?protocols=air_temp_dailies&protocols=humidity_noons&protocols=precipitations&protocols=sky_conditions&startdate=2018-03-12&enddate=2018-03-15&lat=40.71427&lon=-74.00597&distanceinkm=100&geojson=FALSE&sample=FALSE


In [4]:
print(len(date),len(max_temp),len(min_temp),len(precip),len(humidity))
# print(len(max_temp))
# print(len(min_temp))
# print(len(precip))
# print(len(humidity))
print(f"date{date}")
print(f"max_temp{max_temp}")
print(f"min_temp{min_temp}")
print(f"precip{precip}")
print(f"humidity{humidity}")
print(f"latitude{latitude}")
print(f"longitude{longitude}")

45 45 45 45 52
date['2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', nan, nan, nan, '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', nan, nan, nan, nan, '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', nan, '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
max_temp[8.3, 9.3, 10.4, 6.3, nan, nan, nan, 5.9, 4.9, 1.9, 3.1, nan, nan, nan, nan, 6.9, 6.5, 4.5, 6, 8.1, 6.1, 4.6, 6, nan, 7.4, 5.3, 5.2, 7.9, 5.3, 3.9, 2.1, 3.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
min_temp[3, 2.7, 1.9, 0.9, nan, nan, nan, -3.4, -1.4, -2.7, -3.2, nan, nan, nan, nan, -0.8, 0.4, -0.5, 0, -1.3, 0.1, -1, -0.8, nan, -0.9, -2.2, -1.1, -0.8, -3.6, -1.1, -3.2, -2.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
precip['rain', 'rain', 'rain', 'rain', nan, nan, nan, 'no occur

In [ ]:
# print(len(JSON_list))
# print(type(JSON_list[0]))
# pp.pprint(JSON_list[0])
pp.pprint(JSON_list)

## Figure out length of list holding list of JSON response
print(len(JSON_list))   ## length should be number of destinations
print(f"JSON LIST: {type(JSON_list)}")
print(f"JSON LIST FIRST ITEM: {type(JSON_list[0])}")

## figure out length of list holding dicts of API response weather protocols; weather protocols =3 so ##days*3
print(len(JSON_list[0]))     ## length will change according to # days for each city
##each protocol response is in its own dict, so 1 day = 3 weather dict items
print(f"JSON LIST ITEM ONE, FIRST DICT: {type(JSON_list[0][0])}")
# pp.pprint(JSON_list[2])

NOTE: code to test how to extract values from JSON response

In [ ]:
# ## create a variable to hold JSON_list items (i.e, list of each city's historical weather for the date range)
# madrid_list = JSON_list[0]
# # city_list

# ## create variables to hold each day's first 3 weather results (humidity, precip, airtemp)
# madridw_1= madrid_list[0]    ### or JSON_list[0][0]
# madridw_2 = madrid_list[1]
# madridw_3 = madrid_list[2]

# ### create variables to hold each day's second 3 weather results (humidity, precip, airtemp)
# madridw_4= madrid_list[0]
# madridw_5 = madrid_list[1]
# madridw_6 = madrid_list[2] 

# ###etc. etc. over each day in the date range

# # pp.pprint(madridw_1)
# # pp.pprint(JSON_list[0][0])
# # print(madrid_list)
# print(madrid_list[2])
# print(len(madrid_list))


In [ ]:
## initialize dict to hold all output
weather={}


#initialize empty lists to append necessary weather data
date = []
max_temp = []
min_temp = []
precip = []
humidity = []
lat = []
lon= []


## Iterate over madrid_list - first list item of JSON_list (i.e., city1)
i = 0
j = 0

# cityweather_list = JSON_list[i]

## iterate through JSON_list 
for i in range(len(JSON_list)):   ###LIST: len=3 for each city
    cityweather_list = JSON_list[i]
    
    ##iterate through cityweather_list of dict
    for j in range(len(cityweather_list)):     ###LIST: len=n for each day in date range
        
            ## create conditional to pull data from different protocol dicts of cityweather_list
        if cityweather_list[j]['protocol'] == 'air_temp_dailies':
                ### Append JSON response values to lists for each output variable
            date.append(cityweather_list[j]['measuredDate'])
            max_temp.append(cityweather_list[j]['data']['airtempdailiesMaximumTemp'])
            min_temp.append(cityweather_list[j]['data']['airtempdailiesMinimumTemp'])
            lat.append(cityweather_list[j]['latitude'])
            lon.append(cityweather_list[j]['longitude'])
            j=j+1
        elif cityweather_list[j]['protocol'] == 'humidity_noons':
            humidity.append(cityweather_list[j]['data']['humiditynoonsRelativeHumidityPercent'])
            j=j+1
        elif cityweather_list[j]['protocol'] == 'precipitations':
            precip.append(cityweather_list[j]['data']['precipitationsOccurrenceType'])
            j=j+1
        else: 
            date.append(np.nan)
            max_temp.append(np.nan)
            min_temp.append(np.nan)            
            lat.append(np.nan)
            lon.append(np.nan)
            humidity.append(np.nan)
            precip.append(np.nan)
        i=i+1
    
# # for i in range(len(JSON_list)):   ###LIST: len=3 for each city
# cityweather_list = JSON_list[i]
# for j in range(len(cityweather_list)):     ###LIST: len=n for each day in date range
#         ## create conditional to pull data from different weather dicts of results
#     if cityweather_list[j]['protocol'] == 'air_temp_dailies':
#             ### Append JSON response values to lists for each output variable
#         date.append(cityweather_list[j]['measuredDate'])
#         max_temp.append(cityweather_list[j]['data']['airtempdailiesMaximumTemp'])
#         min_temp.append(cityweather_list[j]['data']['airtempdailiesMinimumTemp'])
#         lat.append(cityweather_list[j]['latitude'])
#         lon.append(cityweather_list[j]['longitude'])
#         j=j+1
#     elif cityweather_list[j]['protocol'] == 'humidity_noons':
#         humidity.append(cityweather_list[j]['data']['humiditynoonsRelativeHumidityPercent'])
#         j=j+1
#     elif cityweather_list[j]['protocol'] == 'precipitations':
#         precip.append(cityweather_list[j]['data']['precipitationsOccurrenceType'])
#         j=j+1
#     else: 
#         date.append(np.nan)
#         max_temp.append(np.nan)
#         min_temp.append(np.nan)            
#         lat.append(np.nan)
#         lon.append(np.nan)
#         humidity.append(np.nan)
#         precip.append(np.nan)
#         j=j+1
# j=0
# i=i+1
    
    
    

print(len(date))
print(len(max_temp))
print(len(min_temp))
print(len(precip))
print(len(humidity))
print(date)
print(max_temp)
print(max_temp)
print(min_temp)
print(precip)
print(humidity)
print(lat)
print(lon)


In [ ]:
## Drop NaN values from lists
test = [x for x in date if ~np.isnan(x)]
# date = 
# max_temp = 
# min_temp = 
# precip =
# humidity = 
# lat = 
# lon= 



In [ ]:
##Convert lists into dict (or DF) for output.. ideally append onto weather_params dict at beginning


In [ ]:
#convert data to DataFrame
weather_dict = {"City": destinations,
                           #"Country": country_codes,
                           "Latitude": lats,
                           "Longitude": lons,
                           "Dates": date,
                           "Max Temp": max_temp,
                           "Min Temp": min_temp,
                           "Precipitation": precip_type,
                           "Precipitation Amt": precip,
                           "Humidity": humidity
               }
 

In [ ]:
#### NOTES: could copy the WeatherPy hw and make annual graphs of weather trend for each city/destination
#### line graph of Annual precipitation for Madrid, London, Paris; 
#### error bar graph (max, min, mean) with annual temperatures of Madrid London Paris;
#### bar graph of Annual humidity for Madrid, London Paris